In [ ]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

In [ ]:
html_path = "HTMLs\datascienceweekly.html"
with open(html_path, "r") as f:
    data = BeautifulSoup(f, "html.parser")

In [ ]:
all_uls = data.find_all("ul")
for ul in all_uls:
    previous = ul.previous_sibling.previous_sibling
    if((previous.name == "h2") and ("#34495e" in previous["style"])):
        print(previous.text)
    font = ul.find("font")
    try:
        children = list(font.children)
        
        # print(list(font.children))
        # print(list(font.descendants))
        headline = children[0].text
        
        body = " ".join([x.text for x in children[2:] if "<br" not in x])
        #body = list(font.descendants)[-1]
        # print(headline,"\n--------------------------\n", body)
    except Exception as e:
        children = list(font.children)
        # print([x for x in children[2:] if "<br" not in x])
        # print("ERROR++++++++++++",e,font)